In [1]:
import numpy as np
n_features = 'all'
term_pred = 'long_term'
data_file_name = 'data_after_y10_origin_at_10'
training_id_file_name = 'all_training_ID_outerloop_cohort_10_26'
endpt = 16;
eval_times = 365.25*np.r_[np.arange(2, endpt+1, 2)]
l2_final=0.1
# work_dir = '/home/idies/workspace/Storage/hnguye78/persistent/CARDIA_project/cvd_outcome_rerun2'
work_dir = 'U:/Hieu/CARDIA_project/CARDIA_project/cvd_outcome_rerun_2'

In [10]:
# %pip install tensorflow
# %pip install lifelines

# !y | pip uninstall statsmodels 
# %pip install statsmodels==0.11.0

# %pip install keras
# %pip install theano

  Created wheel for theano: filename=Theano-1.0.5-py3-none-any.whl size=2668111 sha256=46978b0474e895fb754ef957d7456477cf47fa0c733193c9491fa1bfc9ae35cc
Note: you may need to restart the kernel to use updated packages.  Stored in directory: c:\users\hnguye78\appdata\local\pip\cache\wheels\84\cb\19\235b5b10d89b4621f685112f8762681570a9fa14dc1ce904d9
Successfully built theano


In [4]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib
import math
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
from sklearn.preprocessing import StandardScaler
from scipy import stats
import time


In [8]:
snippets_dir



'U:/Hieu/CARDIA_project/CARDIA_project/cvd_outcome_rerun_2/code/snippet'

In [11]:
# snippets_dir = '/home/idies/workspace/Storage/hnguye78/persistent/CARDIA_project/cvd_outcome_rerun'+ '/code/snippet'
snippets_dir = work_dir+'/cardia_rerun_2_code/snippet'
import os, sys
sys.path.append(os.path.abspath(snippets_dir))
import cox_nnet
from cox import *


WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\ProgramData\Anaconda3\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [12]:
load_dir = work_dir+ '/csv_files'

# load data:
data_full = pd.read_csv(load_dir+'/'+data_file_name+'.csv')
data_full = data_full.select_dtypes(include =[np.number])



In [13]:
data_full

,ID,event,time,sex,race,cholesterol,hdl,hbp.medication,smoker,diabetes,...,C08MELAG_YEAR,C08SKNAG_YEAR,C08BRNAG_YEAR,C08STMAG_YEAR,C08OCAAG_YEAR,C08GALDA_YEAR,C08PRGAG_YEAR,C08KYSAG_YEAR,C08URNAG_YEAR,count_na
0,100016012504,0,5258.5,0,0,186.0,38.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.048673
1,100023004268,0,5560.5,0,0,166.0,43.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.055310
2,100033323702,0,5381.5,0,0,234.0,52.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.077434
3,100056526386,0,5511.5,0,1,199.0,44.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.008850
4,100061300991,0,5403.5,0,1,200.0,40.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4195,416761219907,0,5439.5,1,0,225.0,57.0,0,0,0,...,0,0,0,0,0,0,14,0,0,0.048673
4196,416771521620,0,5586.5,0,1,204.0,76.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.004425
4197,416783315386,0,5563.5,1,0,159.0,58.0,0,0,0,...,0,0,0,0,0,0,8,0,0,0.112832
4198,416796224310,0,5452.5,1,1,164.0,55.0,0,0,0,...,0,0,0,0,0,0,14,0,0,0.070796


In [14]:
# load training id:
loaddir = work_dir+ '/csv_files'
trainingid_all = pd.read_csv(loaddir+'/'+training_id_file_name+'.csv')

## standardize feature space, then merge back to label space:
feature_space = data_full.drop(['ID','event','time'], axis = 1)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(feature_space)
scaled_feature_space = scaler.transform(feature_space)
scaled_feature_space_df = pd.DataFrame(data=scaled_feature_space[0:,0:])
scaled_feature_space_df.insert(0, 'ID', data_full['ID'], True)
label = data_full.loc[:,['ID','event','time']]
data_full = pd.merge(label, scaled_feature_space_df)

In [15]:
data_full

,ID,event,time,0,1,2,3,4,5,6,...,444,445,446,447,448,449,450,451,452,453
0,100016012504,0,5258.5,-1.120553,-0.959853,0.239613,-1.105713,-0.123404,1.596965,-0.14629,...,-0.022543,-0.0246,-0.021827,-0.015432,-0.055668,-0.086914,-0.630124,-0.089138,-0.121811,0.774994
1,100023004268,0,5560.5,-1.120553,-0.959853,-0.356702,-0.746751,-0.123404,-0.626188,-0.14629,...,-0.022543,-0.0246,-0.021827,-0.015432,-0.055668,-0.086914,-0.630124,-0.089138,-0.121811,0.981136
2,100033323702,0,5381.5,-1.120553,-0.959853,1.670768,-0.100620,-0.123404,-0.626188,-0.14629,...,-0.022543,-0.0246,-0.021827,-0.015432,-0.055668,-0.086914,-0.630124,-0.089138,-0.121811,1.668274
3,100056526386,0,5511.5,-1.120553,1.041826,0.627217,-0.674959,-0.123404,-0.626188,-0.14629,...,-0.022543,-0.0246,-0.021827,-0.015432,-0.055668,-0.086914,-0.630124,-0.089138,-0.121811,-0.461855
4,100061300991,0,5403.5,-1.120553,1.041826,0.657033,-0.962128,-0.123404,1.596965,-0.14629,...,-0.022543,-0.0246,-0.021827,-0.015432,-0.055668,-0.086914,-0.630124,-0.089138,-0.121811,-0.736711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4195,416761219907,0,5439.5,0.892416,-0.959853,1.402426,0.258341,-0.123404,-0.626188,-0.14629,...,-0.022543,-0.0246,-0.021827,-0.015432,-0.055668,-0.086914,1.966503,-0.089138,-0.121811,0.774994
4196,416771521620,0,5586.5,-1.120553,1.041826,0.776296,1.622396,-0.123404,-0.626188,-0.14629,...,-0.022543,-0.0246,-0.021827,-0.015432,-0.055668,-0.086914,-0.630124,-0.089138,-0.121811,-0.599283
4197,416783315386,0,5563.5,0.892416,-0.959853,-0.565412,0.330134,-0.123404,-0.626188,-0.14629,...,-0.022543,-0.0246,-0.021827,-0.015432,-0.055668,-0.086914,0.853663,-0.089138,-0.121811,2.767696
4198,416796224310,0,5452.5,0.892416,1.041826,-0.416334,0.114757,-0.123404,-0.626188,-0.14629,...,-0.022543,-0.0246,-0.021827,-0.015432,-0.055668,-0.086914,1.966503,-0.089138,-0.121811,1.462133


In [44]:
# Outerloop:
for fold in range(6,25):

    #fold = 24

    print('')
    print('FOLD '+str(fold)+':')
    print('')

    trainingid = trainingid_all.iloc[:,fold]
    trainingid = trainingid[~np.isnan(trainingid)]
    eligible_id = data_full['ID'][data_full['ID'].isin(trainingid)]
    train_df = data_full.loc[data_full['ID'].isin(eligible_id),:]
    del train_df['ID']
    test_df = data_full.loc[~data_full['ID'].isin(eligible_id),:]
    del test_df['ID']


    model_params = dict(node_map = None, input_split = None)
    search_params = dict(method = "nesterov"
                         , learning_rate=0.0001
                         , momentum=0.9
                         , max_iter=200000
                         , stop_threshold=0.995
                         , patience=1000
                         , patience_incr=2
                         , rand_seed = 9114
                         , eval_step=23
                         , lr_decay = 0.9
                         , lr_growth = 1.0)

    # #train final model
    L2_reg = 0.1
    featurespace_train = train_df.drop(['time','event'],axis =1).values
    featurespace_test = test_df.drop(['time','event'],axis =1).values
    model_params = dict(node_map = None, input_split = None, L2_reg=0.1)
    cox_nnet_model, cox_nnet_cost_iter = cox_nnet.trainCoxMlp(featurespace_train
                                                              , train_df.time.values
                                                              , train_df.event.values
                                                              , model_params
                                                              , search_params
                                                              , verbose=True)
    cox_nnet_theta_train = cox_nnet_model.predictNewData(featurespace_train)
    cox_nnet_theta_test = cox_nnet_model.predictNewData(featurespace_test)

    #discrimination on train, test sets
    c_train = concordance_index(train_df.time,-cox_nnet_theta_train, train_df.event)
    print('C on training set:' + str(c_train))
    c_test = concordance_index(test_df.time,-cox_nnet_theta_test, test_df.event)
    print('C on test set:' + str(c_test))

    predicted_risk = cox_nnet_theta_test
    predicted_risk = pd.DataFrame(data=predicted_risk)
    times, H0 = cox_basehaz(cox_nnet_theta_train.astype('float32').flatten(), train_df.time.values, train_df.event.values.astype(bool))
    trained_time = pd.DataFrame(data = times)
    pred_prob_surv = cox_pred_surv(cox_nnet_theta_test.astype('float32').flatten(), H0)
    pred_prob_surv = pd.DataFrame(data=pred_prob_surv)


    savedir = os.path.join(work_dir,'csv_files/cox_nnet/'+str(n_features)+'_features/'+term_pred)
    try: 
        os.makedirs(savedir)
    except OSError:
        if not os.path.isdir(savedir):
            raise
    actual_fold = fold+1

    predicted_risk.to_csv(savedir+'/lp_fold_'+str(actual_fold)+'.csv', index = None, header = True)
    pred_prob_surv.to_csv(savedir+'/pred_prob_surv_fold_'+str(actual_fold)+'.csv', index = None, header = True)
    trained_time.to_csv(savedir+'/trained_time_fold_'+str(actual_fold)+'.csv', index = None, header = True)




FOLD 6:

Using nesterov accelerated gradient
training model
cost: 4.498827, iteration: 0
cost: 4.471642, iteration: 23
cost: 4.434653, iteration: 46
cost: 4.397111, iteration: 69
cost: 4.359835, iteration: 92
cost: 4.322895, iteration: 115
cost: 4.286292, iteration: 138
cost: 4.250025, iteration: 161
cost: 4.214091, iteration: 184
cost: 4.178487, iteration: 207
cost: 4.143210, iteration: 230
cost: 4.108256, iteration: 253
cost: 4.073623, iteration: 276
cost: 4.039308, iteration: 299
cost: 4.005307, iteration: 322
cost: 3.971619, iteration: 345
cost: 3.938240, iteration: 368
cost: 3.905167, iteration: 391
cost: 3.872397, iteration: 414
cost: 3.839929, iteration: 437
cost: 3.807758, iteration: 460
cost: 3.775882, iteration: 483
cost: 3.744299, iteration: 506
cost: 3.713006, iteration: 529
cost: 3.682000, iteration: 552
cost: 3.651278, iteration: 575
cost: 3.620838, iteration: 598
cost: 3.590678, iteration: 621
cost: 3.560795, iteration: 644
cost: 3.531185, iteration: 667
cost: 3.501848,

cost: 0.725109, iteration: 5911
cost: 0.717973, iteration: 5957
cost: 0.710967, iteration: 6003
cost: 0.704090, iteration: 6049
cost: 0.697338, iteration: 6095
cost: 0.690709, iteration: 6141
cost: 0.684202, iteration: 6187
cost: 0.677814, iteration: 6233
cost: 0.671542, iteration: 6279
cost: 0.665385, iteration: 6325
cost: 0.659340, iteration: 6371
cost: 0.653406, iteration: 6417
cost: 0.647580, iteration: 6463
cost: 0.641861, iteration: 6509
cost: 0.636246, iteration: 6555
cost: 0.630734, iteration: 6601
cost: 0.625323, iteration: 6647
cost: 0.620010, iteration: 6693
cost: 0.614795, iteration: 6739
cost: 0.609674, iteration: 6785
cost: 0.604648, iteration: 6831
cost: 0.599713, iteration: 6877
cost: 0.594868, iteration: 6923
cost: 0.590112, iteration: 6969
cost: 0.585443, iteration: 7015
cost: 0.580859, iteration: 7061
cost: 0.576359, iteration: 7107
cost: 0.571941, iteration: 7153
cost: 0.567604, iteration: 7199
cost: 0.563346, iteration: 7245
cost: 0.559166, iteration: 7291
cost: 0.

cost: 1.559832, iteration: 3059
cost: 1.548587, iteration: 3082
cost: 1.537444, iteration: 3105
cost: 1.526404, iteration: 3128
cost: 1.515465, iteration: 3151
cost: 1.504627, iteration: 3174
cost: 1.493888, iteration: 3197
cost: 1.483248, iteration: 3220
cost: 1.472706, iteration: 3243
cost: 1.462260, iteration: 3266
cost: 1.451910, iteration: 3289
cost: 1.441655, iteration: 3312
cost: 1.431494, iteration: 3335
cost: 1.421427, iteration: 3358
cost: 1.411452, iteration: 3381
cost: 1.401568, iteration: 3404
cost: 1.391776, iteration: 3427
cost: 1.382073, iteration: 3450
cost: 1.372459, iteration: 3473
cost: 1.362934, iteration: 3496
cost: 1.353495, iteration: 3519
cost: 1.344144, iteration: 3542
cost: 1.334878, iteration: 3565
cost: 1.325698, iteration: 3588
cost: 1.316602, iteration: 3611
cost: 1.307589, iteration: 3634
cost: 1.298659, iteration: 3657
cost: 1.289811, iteration: 3680
cost: 1.281044, iteration: 3703
cost: 1.272358, iteration: 3726
cost: 1.263751, iteration: 3749
cost: 1.

SystemError: <class 'range'> returned a result with an error set
Apply node that caused the error: Elemwise{Composite{tanh((i0 + i1))}}[(0, 0)](Dot22.0, InplaceDimShuffle{x,0}.0)
Toposort index: 8
Inputs types: [TensorType(float64, matrix), TensorType(float64, row)]
Inputs shapes: [(3360, 22), (1, 22)]
Inputs strides: [(176, 8), (176, 8)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[Dot22(Elemwise{Composite{tanh((i0 + i1))}}[(0, 0)].0, W_cox)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.

In [23]:
# !Conda create -n tf-gpu
# !Conda activate tf-gpu
# !pip install tensorflow-gpu

^C


Could not find conda environment: tf-gpu
You can list all discoverable environments with `conda info --envs`.



In [40]:
os.environ['THEANO_FLAGS']= 'mode=FAST_RUN, device=cuda0, floatX=float32'

In [43]:
from theano import function, config, shared, tensor as tt
import numpy
import time

vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), config.floatX))
f = function([], tt.exp(x))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in range(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
if numpy.any([isinstance(x.op, theano.tensor.elemwise.Elemwise) and
              ('Gpu' not in type(x.op).__name__)
              for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')

[Elemwise{exp,no_inplace}(<TensorType(float64, vector)>)]
Looping 1000 times took 2.456049 seconds
Result is [1.23178032 1.61879341 1.52278065 ... 2.20771815 2.29967753 1.62323285]
Used the cpu


In [42]:
import theano.sandbox.cuda
theano.sandbox.cuda.use('gpu0')

SkipTest: You are importing theano.sandbox.cuda. This is the old GPU back-end and is removed from Theano. Use Theano 0.9 to use it. Even better, transition to the new GPU back-end! See https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

In [28]:
PARAMS = 'THEANO_FLAGS=mode=FAT_RUN.device=gpu0, floatX=float32 %s'
system(sprint(PARAM, 'cox_nnet_long_term_prediction_all_var.ipynb'))

NameError: name 'system' is not defined

In [20]:
os.environ['CUDA_VISIBLE_DEVICES']='1'
tf_device = '/GPU:0'

In [18]:

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17748565743607172743
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 31753514624
locality {
  bus_id: 1
  links {
    link {
      device_id: 1
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 16507535774320947291
physical_device_desc: "device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:25:00.0, compute capability: 7.0"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 31753514624
locality {
  bus_id: 1
  links {
    link {
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 2618526900162166560
physical_device_desc: "device: 1, name: Tesla V100-PCIE-32GB, pci bus id: 0000:c8:00.0, compute capability: 7.0"
]


In [ ]:
# # cross validation:
# from sklearn.model_selection import StratifiedKFold
# n_folds = 10
# kf=StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=0)
# early_stopping = EarlyStopping(monitor='loss', patience=20)

# #l2_array = np.concatenate(([0.],np.power(10.,np.arange(-6,-2))))
# l2_array = np.power(10.,np.arange(-3,2))
# grid_search_train = np.zeros((len(l2_array),n_folds))
# grid_search_test = np.zeros((len(l2_array),n_folds))
# grid_search_c_index_train = np.zeros((len(l2_array),n_folds))
# grid_search_c_index_test = np.zeros((len(l2_array),n_folds))

# for i in range(len(l2_array)):
#     #i = 0
#     j=0
#     cv_folds = kf.split(train_df.values, train_df[['event']].values)
#     for traincv, testcv in cv_folds:
#         print('i in len(l2_array)=' + str(i+1) + '/' + str(len(l2_array)))
#         print('cv fold =' + str(j+1) + '/' + str(n_folds))
#         x_train_cv = featurespace_train_df.iloc[traincv].values
#         y_train_cv = y_train[traincv]
#         x_test_cv = featurespace_train_df.iloc[testcv].values
#         y_test_cv = y_train[testcv]

#         model = Sequential()
#         #model.add(Dense(n_intervals,input_dim=x_train.shape[1],bias_initializer='zeros',kernel_regularizer=regularizers.l2(l2_array[i])))
#         model.add(Dense(hidden_layers_sizes
#                   , input_dim=featurespace_train.shape[1]
#                   , bias_initializer='zeros'
#                   , activation='relu'
#                   , kernel_regularizer=regularizers.l2(l2_array[i])
#                   ))
#         model.add(Dense(n_intervals))
#         model.add(Activation('sigmoid'))
#         model.compile(loss=nnet_survival.surv_likelihood(n_intervals)
#                       , optimizer=optimizers.Adam()) #lr=0.0001))
#         history=model.fit(x_train_cv, y_train_cv
#                     , batch_size=256
#                     , epochs=1000
#                     , callbacks=[early_stopping]
#                     , verbose=1
#                     , validation_data=(x_test_cv, y_test_cv))
#         grid_search_train[i,j] = model.evaluate(x_train_cv,y_train_cv,verbose=0)
#         grid_search_test[i,j] = model.evaluate(x_test_cv,y_test_cv,verbose=0)
              
#         #Discrimination performance
#         # y_pred is conditional prob of survival within each time interval
#         y_pred_train_cv = model.predict_proba(x_train_cv ,verbose=0)
#         yr26_surv_train_cv = nnet_pred_surv(y_pred_train_cv, breaks, 365.25*26)
#         y_pred_test_cv=model.predict_proba(x_test_cv ,verbose=0)
#         yr26_surv_test_cv = nnet_pred_surv(y_pred_test_cv, breaks, 365.25*26)
        
#         grid_search_c_index_train[i,j] = concordance_index(train_df.time.iloc[traincv], yr26_surv_train_cv, train_df.event.iloc[traincv])
#         grid_search_c_index_test[i,j] = concordance_index(train_df.time.iloc[testcv], yr26_surv_test_cv, train_df.event.iloc[testcv])
        
#         print(' ')
#         print('Looking at  i=' + str(i+1) + '/' + str(len(l2_array)) + (' and cv fold =' + str(j+1) + '/' + str(n_folds)))      
#         print('Loss Train, Loss Valid, C-index Train, C-index Test:')
#         print(grid_search_train[i,j])
#         print(grid_search_test[i,j])
#         print(grid_search_c_index_train[i,j])
#         print(grid_search_c_index_test[i,j]) 
#         print(' ')
              
#         j=j+1
        
    


# print('grid_search_train:')
# print(grid_search_train)
# print('grid_search_test:')
# print(grid_search_test)
# print(np.average(grid_search_train,axis=1))
# print(np.average(grid_search_test,axis=1))
# print(grid_search_c_index_train)
# print(grid_search_c_index_test)
#
#l2_final = l2_array[np.argmax(-np.average(grid_search_test,axis=1))]    

In [ ]:

# ##########################
# # Build model with tuned hyperparam:
# l2_final=0.0001

# from numpy.random import seed

# seed(1)
# import tensorflow as tf
# import keras
# tf.random.set_seed(2)

# model = Sequential()
# model.add(Dense(hidden_layers_sizes
#                 , input_dim=featurespace_train.shape[1]
#                 , bias_initializer='zeros'
#                 , kernel_regularizer=regularizers.l2(l2_final)))
# model.add(Activation('relu'))
# model.add(Dense(n_intervals))
# model.add(Activation('sigmoid'))
# # import keras.backend.tensorflow_backend as kk

# model.compile(loss=nnet_survival.surv_likelihood(n_intervals), optimizer=optimizers.RMSprop())
# early_stopping = EarlyStopping(monitor='loss', patience=20)
# history=model.fit(featurespace_train, y_train, batch_size=4063, epochs=100000
#                   , callbacks=[early_stopping]
#                   , verbose=0)


# # print loss of train and valid data:
# # print(model.evaluate(featurespace_train,y_train,verbose=0))
# # model.evaluate(featurespace_test,y_test,verbose=0)

# #Discrimination performance
# # y_pred is conditional prob of survival within each time interval
# y_pred_train = model.predict(featurespace_train,verbose=0)
# # cumprod = cumulative product, the probability of surviving from time 0 up to the time interval of interest
# # index -1 because we are interested in -1 
# last_yr_surv_train=np.cumprod(y_pred_train[:,0:np.nonzero(breaks>365*(endpt-1))[0][0]], axis=1)[:,-1]
# print('Train C-index fold', str(fold+1),':')
# print(concordance_index(train_df.time, last_yr_surv_train, train_df.event)) 

# y_pred=model.predict(featurespace_test,verbose=0)
# last_yr_surv=np.cumprod(y_pred[:,0:np.nonzero(breaks>365*(endpt-1))[0][0]], axis=1)[:,-1]
# print('Test C-index fold', str(fold+1),':')
# print(concordance_index(test_df.time,last_yr_surv, test_df.event))

# pred_surv = np.zeros((len(test_df.event), len(eval_times)))
# col=0
# for time in eval_times:
#     pred_surv[:,col] = nnet_pred_surv(y_pred, breaks, time)
#     col = col+1
# pred_surv = pd.DataFrame(data = pred_surv)
# # savedir = os.path.join(os.getcwd(),'python_files/csv_files/nnet_survival/all_features')
# savedir = os.path.join(work_dir,'csv_files/nnet_survival/'+str(n_features)+'_features/'+term_pred)
# try: 
#     os.makedirs(savedir)
# except OSError:
#     if not os.path.isdir(savedir):
#         raise
# actual_fold = fold+1
# pred_surv.to_csv(savedir+'/pred_prob_surv_fold_'+str(actual_fold)+'.csv', index = None, header = True)


